In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization

# Dataset Example

In [2]:
BASE_DIR = os.path.abspath(os.path.join("cats-and-dogs", "training_set"))
CATS_DIR = os.path.join(BASE_DIR, "cats")
DOGS_DIR = os.path.join(BASE_DIR, "dogs")

BATCH_SIZE = 16
IMAGE_SIZE = (128, 128)

In [3]:
cat_images = [os.path.join(CATS_DIR, f) for f in os.listdir(CATS_DIR)]
dog_images = [os.path.join(DOGS_DIR, f) for f in os.listdir(DOGS_DIR)]

In [4]:
all_images = cat_images + dog_images

In [14]:
len(all_images)

8000

In [5]:
def preprocess_images(filename):
    file = tf.io.read_file(filename)
    
    img = tf.image.decode_jpeg(file)
    img = tf.image.resize(img, IMAGE_SIZE)
    img = img / 255.0
    
    label = tf.strings.split(filename, sep = "\\")[-2] == "cats"
    label = tf.cast(label, tf.int32)
    
    return img, label

In [6]:
dataset = tf.data.Dataset.from_tensor_slices(all_images) \
    .shuffle(len(all_images)) \
    .map(preprocess_images) \
    .batch(BATCH_SIZE) \
    .prefetch(tf.data.AUTOTUNE) \
    .repeat()

In [19]:
for x in dataset:
    print(x)
    break

(<tf.Tensor: shape=(16, 128, 128, 3), dtype=float32, numpy=
array([[[[9.02743459e-01, 9.06665027e-01, 8.87057185e-01],
         [8.63126636e-01, 8.67048204e-01, 8.47440362e-01],
         [9.25817132e-01, 9.29738700e-01, 9.10130858e-01],
         ...,
         [8.61706793e-01, 8.42344046e-01, 8.37809741e-01],
         [9.68528867e-01, 9.60685670e-01, 9.64607298e-01],
         [9.88387048e-01, 9.82458234e-01, 9.86379802e-01]],

        [[9.68229175e-01, 9.72150743e-01, 9.52542901e-01],
         [8.73958349e-01, 8.77879918e-01, 8.58272076e-01],
         [8.81500363e-01, 8.85421932e-01, 8.65814090e-01],
         ...,
         [9.43202853e-01, 9.23840106e-01, 9.19305801e-01],
         [9.65883017e-01, 9.58039880e-01, 9.61961448e-01],
         [9.39849496e-01, 9.32006359e-01, 9.35927927e-01]],

        [[9.57061887e-01, 9.53140318e-01, 9.37454045e-01],
         [9.17360306e-01, 9.13438737e-01, 8.97752464e-01],
         [9.09809172e-01, 9.05887604e-01, 8.90201330e-01],
         ...,
         

In [8]:
model = Sequential([
    Input((128, 128, 3)),
    Conv2D(filters = 64, kernel_size = (5, 5), padding = "same", activation = "relu"),
    Conv2D(filters = 64, kernel_size = (5, 5), padding = "same", activation = "relu"),
    MaxPool2D(),
    BatchNormalization(),
    Conv2D(filters = 32, kernel_size = (3, 3), padding = "same", activation = "relu"),
    Conv2D(filters = 32, kernel_size = (3, 3), padding = "same", activation = "relu"),
    MaxPool2D(),
    BatchNormalization(),
    Conv2D(filters = 16, kernel_size = (3, 3), padding = "same", activation = "relu"),
    Conv2D(filters = 16, kernel_size = (3, 3), padding = "same", activation = "relu"),
    MaxPool2D(),
    Flatten(),
    Dense(64, activation = "relu"),
    Dropout(0.2),
    Dense(32, activation = "relu"),
    Dropout(0.1),
    Dense(1, activation = "sigmoid")
])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 64)      4864      
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 64)      102464    
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 64)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 64, 64, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 32)        18464     
                                                                 
 conv2d_3 (Conv2D)           (None, 64, 64, 32)        9

In [10]:
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [11]:
model.fit(dataset, steps_per_epoch = 500, epochs = 4)

Epoch 1/4
500/500 [==============================] - 261s 431ms/step - loss: 0.6808 - accuracy: 0.5748
Epoch 2/4
500/500 [==============================] - 256s 512ms/step - loss: 0.6275 - accuracy: 0.6564
Epoch 3/4
 90/500 [====>.........................] - ETA: 3:00 - loss: 0.5728 - accuracy: 0.7069

InternalError: Could not synchronize CUDA stream: CUDA_ERROR_LAUNCH_FAILED: unspecified launch failure